<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/Sentences_SimilaritiesEspanol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gensim
!pip install -U pip setuptools wheel
#pip install -U spacy

!python -m spacy download es_core_news_sm
!spacy download es_core_news_sm
#!python -m spacy download es_core_news_md
#!spacy download es_core_news_md

  Using cached gensim-3.8.3-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
     |████████████████████████████████| 24.2 MB 54 kB/s 
  Using cached pip-21.0-py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 1.5 MB 5.2 MB/s 
  Using cached setuptools-52.0.0-py3-none-any.whl (784 kB)
     |████████████████████████████████| 786 kB 18.1 MB/s 
     |████████████████████████████████| 16.2 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
     |████████████████████████████████| 16.2 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [21]:
from gensim.models import Word2Vec
import spacy
print(spacy.__version__)
nlp = spacy.load('es_core_news_sm')
news_headlines= []


frases=[["escribir","tutoriales","es","gratificante"],["pero","comer","papas","bravas","es","mucho","mejor"]]
news_headline0 = "Amo el futbol"
news_headline1 = "Futbol es mi amor"
news_headline2 = "El mejor deporte es el futbol"
news_headline3 = "Sopa de pollo es lo mejor que hay"
news_headlines_withoutLemma = [news_headline0 ,news_headline1,news_headline2, news_headline3]
#https://unipython.com/curso-de-procesamiento-de-textos-gensim/

2.2.4


In [16]:
modelo = Word2Vec(frases,min_count=1)


In [22]:
from gensim.models.keyedvectors import KeyedVectors
 
modelo = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/GensimSpanish/sbw_vectors.bin", binary=True)

In [23]:
# Python program to remove punctuation from a given string 
# Function to remove punctuation 
def Punctuation(string): 
    string= string.replace("-PRON-","")      
  
    # punctuation marks 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~-'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
  
    # Print string without punctuation 
    return (string) 
#Lemmatizing from beggning
for headlines in news_headlines_withoutLemma:
   doc = nlp(Punctuation(headlines))
   line = " ".join([token.lemma_ for token in doc])
   line= line.replace("-PRON-","") 
   line= line.replace("  "," ")      
     

   print(line)
   news_headlines.append(line)
print(news_headlines)

Amo el futbol
Futbol ser mi amor
El mejor deportar ser el futbol
Sopa de pollo ser el mejor que haber
['Amo el futbol', 'Futbol ser mi amor', 'El mejor deportar ser el futbol', 'Sopa de pollo ser el mejor que haber']


In [24]:
headline_tokens = []
for news_headline in news_headlines:
    headline_tokens.append([token.text.lower() for token in nlp(news_headline) if not token.is_stop])

print(headline_tokens)

[['amo', 'futbol'], ['futbol', 'amor'], ['deportar', 'futbol'], ['sopa', 'pollo']]


In [25]:
subject_headline = news_headlines[0]
subject_token = headline_tokens[0]

print('Headline: ', subject_headline)
print('=' * 50)
print()
lineNumber = 0
for token, headline in zip(headline_tokens, news_headlines):
    print('-' * 50)
    print(token)
    print('Comparing to:', news_headlines_withoutLemma[lineNumber])
    lineNumber += 1
    distance = modelo.wmdistance(subject_token, token)
    print('distance = %.4f' % distance)

Headline:  Amo el futbol

--------------------------------------------------
['amo', 'futbol']
Comparing to: Amo el futbol
distance = 0.0000
--------------------------------------------------
['futbol', 'amor']
Comparing to: Futbol es mi amor
distance = 1.4893
--------------------------------------------------
['deportar', 'futbol']
Comparing to: El mejor deporte es el futbol
distance = 2.4910
--------------------------------------------------
['sopa', 'pollo']
Comparing to: Sopa de pollo es lo mejor que hay
distance = 5.0148
